In [ ]:
import re
import os
import subprocess
import template

class CodeProcessor:
    
    def __init__(self):
        self.code_script = None
        self.code_save_dir = "/path/to/save/dir"  # 假定保存代码的目录

    def extract_code_blocks(self, response):
        # 正则表达式匹配代码块
        code_pattern = re.compile(r'```(?:\w+)?\n(.*?)```', re.DOTALL)
        code_blocks = code_pattern.findall(response)
        non_code_content = code_pattern.sub('', response).strip()
        self.code_script = '\n'.join(code_blocks)  # 确保是一个字符串
        return code_blocks, non_code_content

    def get_all_prompts_for_log(self):
        # 伪造数据以记录提示格式
        data = SpoofData()  # 伪造数据的实例化
        code_blocks, _ = self.extract_code_blocks(data)
        prompts = dict(
            gen_ini_task_decription=self.gen_ini_task_decription(data),
            task_step_list=self.manual_modify_step(data),
            step_code=code_blocks
        )
        return prompts

    def run_script(self, input_data, step_id, output_suffix):
        code = self.code_script
        script_dir = self.code_save_dir
        file_path = os.path.join(script_dir, f'{step_id}_script.py')
        output_file_path = os.path.join(script_dir, f'{step_id}_output.{output_suffix}')
        
        try:
            with open(file_path, "w") as file:
                file.write(code)
            log_w_indent(f"Code script saved to {file_path}", 1, "Code")
        except Exception as e:
            log_w_indent(f"Error writing to file: {e}", 1, "Code")
            return f"Error writing to file: {e}"

        cmd_command = f"python {file_path} {input_data}"
        try:
            result = subprocess.run(cmd_command, shell=True, capture_output=True, text=True)
            result.check_returncode()  # 检查命令是否成功运行
            log_w_indent(f"Code script ran successfully!! {file_path}:{result.stdout}", 1, "Code")
            return result.stdout
        except subprocess.CalledProcessError as e:
            log_w_indent(f"Error running script: {e.stderr.strip()}", 1, "Code")
            return e.stderr.strip()
        except Exception as e:
            log_w_indent(f"Unexpected error: {str(e)}", 1, "Code")
            return str(e)
        
    #def pass_metric()

processor = CodeProcessor()
response = """..."""  # 你的输入内容
processor.extract_code_blocks(response)
processor.get_all_prompts_for_log()
processor.pass_metric
processor.run_script("input_data", "step_id", "txt")
